In [0]:
# import libraries

import pandas as pd
import pickle
import numpy as np
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests


!pip install seaborn==0.9.0
import seaborn as sns

In [0]:
pickle_abundance_info = open("varianceDF.pickle","rb")
pickle_protein_to_complex = open("BrentsComplexToProteins.pickle","rb")
abundance_info = pickle.load(pickle_abundance_info)
len(abundance_info)
#abundance_info = abundance_info.set_index("Protein IDs", drop = False)
abundance_info = abundance_info.loc[:, abundance_info.columns.str.startswith('LFQ')]
protein_to_complex = pickle.load(pickle_protein_to_complex)
#protein_to_complex
abundance_info


In [0]:
# make a list of the column names that represent healthy / cancerous samples

healthyColumnNames = []
cancerColumnNames = []
for column in abundance_info.columns:
    if "_NM" in column:
        healthyColumnNames.append(column)
    else:
        cancerColumnNames.append(column)

In [0]:
healthy_abundance = {}
sick_abundance = {}
cancer_div_by_healthy_abundance_ratio = {}






In [0]:
for index, row in abundance_info.iterrows():
  healthy_list = row[healthyColumnNames]
  sick_list = row[cancerColumnNames]
  #print(healthy_list)
  #print(sick_list)
  #Remove Values 0 or close
  cleanHealthyDataList = np.where(np.isclose(healthy_list,0), np.nan, healthy_list)
  cleanHealthyDataList = [x for x in cleanHealthyDataList if str(x) != 'nan']
  #Remove Values 0 or close
  cleanSickDataList = np.where(np.isclose(sick_list,0), np.nan, sick_list)
  cleanSickDataList = [x for x in cleanSickDataList if str(x) != 'nan']
  #Get Average Abundance Levels
  protein_name = index
  avg_cancer_abundance = np.mean(cleanSickDataList)
  avg_healthy_abundance = np.mean(cleanHealthyDataList)
  healthy_abundance[protein_name] = avg_healthy_abundance
  sick_abundance[protein_name] = avg_cancer_abundance
  cancer_div_by_healthy_abundance_ratio[protein_name] = avg_cancer_abundance/avg_healthy_abundance
  

In [0]:
healthy_abundance

In [0]:
sick_abundance

In [0]:
cancer_div_by_healthy_abundance_ratio

In [0]:
protein_to_complex

In [0]:
finalDict = {}

for key,values in protein_to_complex.items():
    tempArray = []
    for value in values:
        if value in cancer_div_by_healthy_abundance_ratio:
            tempArray.append(cancer_div_by_healthy_abundance_ratio[value])
            #print(cancer_div_by_healthy_abundance_ratio[value])
    finalDict[key] = tempArray

totalCount = 0
for key, value in finalDict.items():
    if len(value) != 0:
        totalCount += 1

print(totalCount)

2333


In [0]:
finalDict
abundanceRatiosPickle = open("cancer_abnd_div_by_healthy_dict.pickle", "wb")
pickle.dump(finalDict,abundanceRatiosPickle)